# Model Explainability with SHAP

This notebook demonstrates how to use SHAP (SHapley Additive exPlanations) to explain the predictions of our telecom revenue optimization models. SHAP provides consistent, locally accurate feature importance values that sum up to the difference between the expected model output and the current model output.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# For SHAP
import shap

# For models
import lightgbm as lgb
import xgboost as xgb
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, roc_auc_score, mean_squared_error

# Set plotting style
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

print("Libraries imported successfully!")

## 1. Load and Prepare Data

Let's load our processed telecom customer data for model explainability analysis.

In [ ]:
# Load the master dataset
df = pd.read_csv('../data/raw/master_dataset.csv')
print(f"Dataset shape: {df.shape}")
print(f"Columns: {list(df.columns)}")

## 2. Churn Prediction Model Explainability

Let's first explain our churn prediction model using SHAP.

In [ ]:
# Prepare features for churn prediction
churn_features = [
    'age', 'annual_income', 'tenure_months', 'arpu', 'satisfaction_score',
    'monthly_data_gb', 'monthly_minutes', 'ott_usage_hours',
    'monthly_web_sessions', 'monthly_app_sessions', 'self_service_transactions',
    'num_complaints_12m', 'support_tickets_12m', 'late_payments_12m',
    'campaigns_exposed', 'total_clicks', 'total_conversions'
]

# Create additional features
df['risk_score'] = (
    (10 - df['satisfaction_score']) * 0.4 +
    df['num_complaints_12m'] * 0.3 +
    df['late_payments_12m'] * 0.3
)

df['usage_efficiency'] = (
    df['monthly_data_gb'] / (df['data_allowance_gb'] + 1e-6) +
    df['monthly_minutes'] / (df['minutes_allowance'] + 1e-6)
) / 2

df['value_score'] = pd.qcut(df['arpu'], q=5, labels=[1,2,3,4,5]).astype(int)

# Add enhanced features
churn_features.extend(['risk_score', 'usage_efficiency', 'value_score'])

X_churn = df[churn_features]
y_churn = df['churned']

print(f"Churn prediction features: {len(churn_features)}")
print(f"Feature columns: {churn_features}")

In [ ]:
# Split data
X_train_churn, X_test_churn, y_train_churn, y_test_churn = train_test_split(
    X_churn, y_churn, test_size=0.2, random_state=42, stratify=y_churn
)

print(f"Training set: {X_train_churn.shape}")
print(f"Test set: {X_test_churn.shape}")

In [ ]:
# Train churn prediction model (LightGBM)
print("Training LightGBM churn prediction model...")

churn_model = lgb.LGBMClassifier(
    n_estimators=200,
    max_depth=8,
    learning_rate=0.1,
    subsample=0.8,
    random_state=42,
    verbose=-1
)

churn_model.fit(X_train_churn, y_train_churn)

# Evaluate
y_pred_churn = churn_model.predict(X_test_churn)
y_prob_churn = churn_model.predict_proba(X_test_churn)[:, 1]
auc_churn = roc_auc_score(y_test_churn, y_prob_churn)

print(f"Churn model AUC: {auc_churn:.4f}")

## 3. SHAP Analysis for Churn Model

Now let's use SHAP to explain the churn prediction model.

In [ ]:
# Create SHAP explainer for churn model
print("Creating SHAP explainer for churn model...")

# Use TreeExplainer for tree-based models
churn_explainer = shap.TreeExplainer(churn_model)

# Calculate SHAP values for a sample of test data (for performance)
sample_indices = np.random.choice(X_test_churn.shape[0], size=1000, replace=False)
X_sample = X_test_churn.iloc[sample_indices]

print(f"Calculating SHAP values for {X_sample.shape[0]} samples...")
churn_shap_values = churn_explainer.shap_values(X_sample)

print("SHAP analysis completed!")

In [ ]:
# Summary plot of feature importance
plt.figure(figsize=(12, 10))
shap.summary_plot(churn_shap_values[1], X_sample, show=False)
plt.title('SHAP Feature Importance - Churn Prediction Model', fontsize=16)
plt.tight_layout()
plt.show()

In [ ]:
# Bar plot of mean absolute SHAP values
plt.figure(figsize=(12, 8))
shap.summary_plot(churn_shap_values[1], X_sample, plot_type="bar", show=False)
plt.title('Mean Absolute SHAP Values - Churn Prediction Model', fontsize=16)
plt.tight_layout()
plt.show()

In [ ]:
# Detailed analysis of top features
print("=== TOP FEATURES INFLUENCING CHURN PREDICTIONS ===")

# Calculate mean absolute SHAP values
feature_names = X_sample.columns
mean_abs_shap = np.abs(churn_shap_values[1]).mean(0)

# Create DataFrame and sort
shap_df = pd.DataFrame({
    'feature': feature_names,
    'mean_abs_shap': mean_abs_shap
}).sort_values('mean_abs_shap', ascending=False)

print("Top 10 Most Important Features:")
for i, row in shap_df.head(10).iterrows():
    print(f"  {row['feature']}: {row['mean_abs_shap']:.4f}")

## 4. Individual Prediction Explanations

Let's examine individual customer predictions to understand how features contribute to specific outcomes.

In [ ]:
# Select a few high-risk and low-risk customers for detailed analysis
probabilities = churn_model.predict_proba(X_sample)[:, 1]

# High-risk customer (top 5%)
high_risk_idx = np.argsort(probabilities)[-1]
high_risk_customer = X_sample.iloc[high_risk_idx:high_risk_idx+1]

# Low-risk customer (bottom 5%)
low_risk_idx = np.argsort(probabilities)[0]
low_risk_customer = X_sample.iloc[low_risk_idx:low_risk_idx+1]

print(f"High-risk customer churn probability: {probabilities[high_risk_idx]:.3f}")
print(f"Low-risk customer churn probability: {probabilities[low_risk_idx]:.3f}")

In [ ]:
# Explain high-risk customer prediction
print("=== HIGH-RISK CUSTOMER EXPLANATION ===")
shap.initjs()
high_risk_shap = churn_explainer.shap_values(high_risk_customer)
shap.force_plot(churn_explainer.expected_value[1], high_risk_shap[1], high_risk_customer)

In [ ]:
# Explain low-risk customer prediction
print("=== LOW-RISK CUSTOMER EXPLANATION ===")
low_risk_shap = churn_explainer.shap_values(low_risk_customer)
shap.force_plot(churn_explainer.expected_value[1], low_risk_shap[1], low_risk_customer)

## 5. ARPU Prediction Model Explainability

Let's now explain our ARPU prediction model using SHAP.

In [ ]:
# Prepare features for ARPU prediction
arpu_features = [
    'age', 'annual_income', 'tenure_months', 'satisfaction_score',
    'monthly_data_gb', 'monthly_minutes', 'ott_usage_hours',
    'monthly_web_sessions', 'monthly_app_sessions',
    'num_complaints_12m', 'support_tickets_12m', 'late_payments_12m'
]

X_arpu = df[arpu_features]
y_arpu = df['arpu']

print(f"ARPU prediction features: {len(arpu_features)}")

In [ ]:
# Split data
X_train_arpu, X_test_arpu, y_train_arpu, y_test_arpu = train_test_split(
    X_arpu, y_arpu, test_size=0.2, random_state=42
)

print(f"Training set: {X_train_arpu.shape}")
print(f"Test set: {X_test_arpu.shape}")

In [ ]:
# Train ARPU prediction model (Random Forest)
print("Training Random Forest ARPU prediction model...")

arpu_model = RandomForestRegressor(
    n_estimators=100,
    max_depth=10,
    random_state=42,
    n_jobs=-1
)

arpu_model.fit(X_train_arpu, y_train_arpu)

# Evaluate
y_pred_arpu = arpu_model.predict(X_test_arpu)
rmse_arpu = np.sqrt(mean_squared_error(y_test_arpu, y_pred_arpu))

print(f"ARPU model RMSE: ${rmse_arpu:.2f}")

## 6. SHAP Analysis for ARPU Model

Now let's use SHAP to explain the ARPU prediction model.

In [ ]:
# Create SHAP explainer for ARPU model
print("Creating SHAP explainer for ARPU model...")

# Use TreeExplainer for tree-based models
arpu_explainer = shap.TreeExplainer(arpu_model)

# Calculate SHAP values for a sample of test data
sample_indices_arpu = np.random.choice(X_test_arpu.shape[0], size=1000, replace=False)
X_sample_arpu = X_test_arpu.iloc[sample_indices_arpu]

print(f"Calculating SHAP values for {X_sample_arpu.shape[0]} samples...")
arpu_shap_values = arpu_explainer.shap_values(X_sample_arpu)

print("SHAP analysis completed!")

In [ ]:
# Summary plot of feature importance
plt.figure(figsize=(12, 10))
shap.summary_plot(arpu_shap_values, X_sample_arpu, show=False)
plt.title('SHAP Feature Importance - ARPU Prediction Model', fontsize=16)
plt.tight_layout()
plt.show()

In [ ]:
# Bar plot of mean absolute SHAP values
plt.figure(figsize=(12, 8))
shap.summary_plot(arpu_shap_values, X_sample_arpu, plot_type="bar", show=False)
plt.title('Mean Absolute SHAP Values - ARPU Prediction Model', fontsize=16)
plt.tight_layout()
plt.show()

In [ ]:
# Detailed analysis of top features
print("=== TOP FEATURES INFLUENCING ARPU PREDICTIONS ===")

# Calculate mean absolute SHAP values
feature_names_arpu = X_sample_arpu.columns
mean_abs_shap_arpu = np.abs(arpu_shap_values).mean(0)

# Create DataFrame and sort
shap_df_arpu = pd.DataFrame({
    'feature': feature_names_arpu,
    'mean_abs_shap': mean_abs_shap_arpu
}).sort_values('mean_abs_shap', ascending=False)

print("Top 10 Most Important Features:")
for i, row in shap_df_arpu.head(10).iterrows():
    print(f"  {row['feature']}: {row['mean_abs_shap']:.4f}")

## 7. Uplift Model Explainability

Let's explain our uplift modeling for cross-sell/up-sell opportunities.

In [ ]:
# Prepare features for uplift modeling
uplift_features = ['age', 'annual_income', 'tenure_months', 'satisfaction_score',
                  'monthly_data_gb', 'arpu', 'risk_score']

# Create treatment variable (campaign responsive)
df['treatment'] = (df['total_conversions'] > 0).astype(int)
df['outcome'] = (df['total_conversions'] > 0).astype(int)  # Binary outcome

X_uplift = df[uplift_features]
treatment = df['treatment']
outcome = df['outcome']

print(f"Uplift modeling features: {len(uplift_features)}")

In [ ]:
# Split data
X_train_uplift, X_test_uplift, treatment_train, treatment_test, outcome_train, outcome_test = train_test_split(
    X_uplift, treatment, outcome, test_size=0.2, random_state=42
)

print(f"Training set: {X_train_uplift.shape}")
print(f"Test set: {X_test_uplift.shape}")

In [ ]:
# Train uplift models (two-model approach)
print("Training uplift models...")

# Split into treatment and control groups
treatment_mask = treatment_train == 1
control_mask = treatment_train == 0

X_treatment = X_train_uplift[treatment_mask]
y_treatment = outcome_train[treatment_mask]

X_control = X_train_uplift[control_mask]
y_control = outcome_train[control_mask]

# Train separate models
treatment_model = RandomForestClassifier(n_estimators=100, random_state=42)
control_model = RandomForestClassifier(n_estimators=100, random_state=42)

treatment_model.fit(X_treatment, y_treatment)
control_model.fit(X_control, y_control)

print("Uplift models training completed!")

## 8. SHAP Analysis for Uplift Models

Let's use SHAP to explain both treatment and control models.

In [ ]:
# Create SHAP explainers for uplift models
print("Creating SHAP explainers for uplift models...")

# Treatment model explainer
treatment_explainer = shap.TreeExplainer(treatment_model)

# Control model explainer
control_explainer = shap.TreeExplainer(control_model)

# Sample data for SHAP analysis
sample_indices_uplift = np.random.choice(X_test_uplift.shape[0], size=500, replace=False)
X_sample_uplift = X_test_uplift.iloc[sample_indices_uplift]

print(f"Calculating SHAP values for {X_sample_uplift.shape[0]} samples...")

# Calculate SHAP values
treatment_shap_values = treatment_explainer.shap_values(X_sample_uplift)
control_shap_values = control_explainer.shap_values(X_sample_uplift)

print("SHAP analysis completed!")

In [ ]:
# Summary plot for treatment model
plt.figure(figsize=(12, 10))
shap.summary_plot(treatment_shap_values[1], X_sample_uplift, show=False)
plt.title('SHAP Feature Importance - Treatment Model (Uplift)', fontsize=16)
plt.tight_layout()
plt.show()

In [ ]:
# Summary plot for control model
plt.figure(figsize=(12, 10))
shap.summary_plot(control_shap_values[1], X_sample_uplift, show=False)
plt.title('SHAP Feature Importance - Control Model (Uplift)', fontsize=16)
plt.tight_layout()
plt.show()

## 9. Business Insights from SHAP Analysis

Let's extract business insights from our SHAP explainability analysis.

In [ ]:
print("=== BUSINESS INSIGHTS FROM SHAP ANALYSIS ===")

print("\n1. CHURN PREDICTION INSIGHTS:")
print("   Key drivers of churn include:")
for i, row in shap_df.head(5).iterrows():
    print(f"   - {row['feature']} (importance: {row['mean_abs_shap']:.4f})")

print("\n2. ARPU PREDICTION INSIGHTS:")
print("   Key drivers of revenue include:")
for i, row in shap_df_arpu.head(5).iterrows():
    print(f"   - {row['feature']} (importance: {row['mean_abs_shap']:.4f})")

print("\n3. ACTIONABLE RECOMMENDATIONS:")
print("   For Churn Prevention:")
print("   - Focus on customers with high risk scores")
print("   - Proactively address satisfaction issues")
print("   - Monitor payment behavior closely")

print("\n   For Revenue Optimization:")
print("   - Target high-value customers for premium services")
print("   - Encourage efficient usage patterns")
print("   - Leverage digital engagement channels")

print("\n   For Campaign Targeting:")
print("   - Use uplift models to identify persuadable customers")
print("   - Personalize offers based on customer profiles")
print("   - Optimize marketing spend on high-uplift segments")

## 10. Model Governance and Compliance

SHAP explanations help ensure our models are transparent and compliant with regulations.

In [ ]:
print("=== MODEL GOVERNANCE AND COMPLIANCE ===")

print("1. REGULATORY COMPLIANCE:")
print("   - GDPR: Transparent decision-making for customer data processing")
print("   - CCPA: Right to explanation for automated decisions")
print("   - Financial Regulations: Model interpretability for risk assessment")

print("\n2. ETHICAL AI PRACTICES:")
print("   - Fairness: Identifying potential bias in feature importance")
print("   - Accountability: Clear audit trail of model decisions")
print("   - Transparency: Stakeholder understanding of AI systems")

print("\n3. BUSINESS VALUE:")
print("   - Trust: Increased confidence in AI-driven decisions")
print("   - Optimization: Data-driven insights for business strategy")
print("   - Innovation: Safe exploration of new opportunities")

print("\nModel explainability with SHAP successfully implemented!")
print("This framework enables transparent, accountable, and effective AI deployment.")